In [ ]:
import numpy as np
import pandas as pd
#modelo
from sklearn.preprocessing import MinMaxScaler
from keras.datasets.fashion_mnist import load_data
from keras.models import Model
from keras.layers import Dense, BatchNormalization, LeakyReLU, Conv2DTranspose, Conv2D, Dropout, Flatten, Reshape, Input, Lambda, Concatenate
from keras.layers import LSTM, Dropout, Dense
from keras.models import Sequential
import tensorflow as tf
import scipy
import optuna
from optkeras.optkeras import OptKeras
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_percentage_error, median_absolute_error
import warnings
warnings.filterwarnings("ignore")

### RNN: Model
#### Keras es una biblioteca de redes neuronales de alto nivel, escrita en Python y capaz de ejecutarse sobre TensorFlow
Keras Sequential: para inicializar la red neuronal, crea el contenedor de la red lstm. Este modelo se refiere a que crearemos una serie de capas de neuronas secuenciales, “una delante de otra”.
Dropout : para evitar el sobreajuste con capas de deserción (Especificar 0.2 en la capa Dropout significa que se eliminarán el 20% de las capas)
Dense: para agregar una capa de red neuronal densamente conectada.

La capa LSTM se agrega con los siguientes argumentos:
-num:numero de neuronas
-return_sequences: True es necesario para apilar capas LSTM, por lo que la capa LSTM consecuente tiene una entrada de secuencia tridimensional
-input_shape: es la forma del entrenamiento conjunto de datos.

In [1]:
def create_LSTM_RNN(num,epochs_rnn,batch_size_rnn,X_train,y_train,X_test,y_test):
    
    '''La red tiene una capa visible con 1 entrada, 4 capas ocultas con 50 bloques o neuronas y una capa de
       salida que hace la predicción de un valor único. La red está entrenada con 300 epocas y se utiliza 
       tamaño de lote de 32'''
    
    model = Sequential() #crea el contenedor de la red lstm
    #first LSTM
    model.add(LSTM(units=num,return_sequences=True,input_shape= X_train.shape[1:])) # se agrega la red especificandoles el numero de neuronas:50 y el tamaño de la entrada:35,1
    model.add(Dropout(0.2)) # regularizacion en las conexiones de entrada
    #second LSTM
    model.add(LSTM(units=num,return_sequences=True))
    model.add(Dropout(0.2))
    #third LSTM
    model.add(LSTM(units=num,return_sequences=True))
    model.add(Dropout(0.2))
    #fourth LSTM
    model.add(LSTM(num))
    model.add(Dropout(0.2))

    #Output 
    model.add(Dense(units=1)) #activation='relu'    # para la capa de salida usamos la funcion dense y especificamos que el dato de salida tendra tamaño igual a 1
    
    '''Para compilar nuestro modelo usamos el optimizador de Adam y establecemos la pérdida como mean_squared_error. 
    Después de eso, ajustamos el modelo para que se ejecute durante 300 épocas (las épocas son la cantidad de veces 
    que el algoritmo de aprendizaje funcionará en todo el conjunto de entrenamiento) con un tamaño de lote de 32.'''
    
    model.compile(optimizer='adam',loss='mse', metrics=[tf.metrics.MeanAbsoluteError(),tf.metrics.MeanAbsolutePercentageError()])
    
    #definir "early stopping" la loss de validacion no mejora en X épocas
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss',
                                                      patience=200,  #numero de epocas sin mejora
                                                      restore_best_weights = True)
    
    # Se ajusta el modelo
    history_rnn = model.fit(
    X_train,
    y_train,
    batch_size = batch_size_rnn, 
    verbose = 1,
    epochs = epochs_rnn, 
    validation_data = (X_test, y_test)
    ,callbacks=[early_stopping])
    
    # Función de perdida
    print('Función de perdida de acuerdo al número de epocas')
    plt.plot(history_rnn.history['loss'])
    plt.show()
    
    return(model)

In [2]:
def main(df):
    
    data = df.copy()
    
    # Train - Test
    data_train, data_test = train_test_split(data)
    print('Shape data train:', data_train.shape) 
    print('Shape data test:', data_test.shape) 
    
    # Normalizar datos (Reshape da forma a una matriz)
    scaler, data_train_scaled, data_test_scaled = normalize(data_train.values.reshape (-1, 1), data_test.values.reshape (-1, 1))
     
    # Estructura data para input rnn
    time_step_in = 35 # predecir un valor en el futuro a partir de 35 valores pasados
    X_train, y_train = processData_3D_structure(data_train_scaled, time_step_in) 
    X_test, y_test= processData_3D_structure(data_test_scaled, time_step_in) 
    
    print('\nEstructura 3D : Input RNN')
    print('Dimension X_train',X_train.shape )
    print('Dimension y_train',y_train.shape )
    print('\nDimension X_test',X_test.shape )
    print('Dimension y_test',y_test.shape )
    
    # model LSTM
    
    #Hiperparametros rnn
    batch_size_rnn= 32 # es el número de muestras entre las actualizaciones de peso del modelo
    epochs_rnn= 500 # repeticiones
    num=50 # número de neuronas

    print('\nComenzando con el entrenamiento de la red..')
    model=create_LSTM_RNN(num,epochs_rnn,batch_size_rnn,X_train,y_train,X_test,y_test)
    
    print('\nTermino del entrenamiento')
    
    # Estimamos el rendimiento del modelo para el conjunto de datos de entrenamiento y prueba
    predict_train_rnn= model.predict(X_train)
    y_train_predicted_rnn= scaler.inverse_transform(predict_train_rnn)
    y_train_real=scaler.inverse_transform(y_train)

    predict_test_rnn= model.predict(X_test)
    y_test_predicted_rnn= scaler.inverse_transform(predict_test_rnn)
    y_test_real=scaler.inverse_transform(y_test)

    prueba_y_predicted=np.concatenate((y_train_predicted_rnn, y_test_predicted_rnn), axis=0, out=None)
    prueba_y_real=np.concatenate((y_train_real, y_test_real), axis=0, out=None)
    
    print('Rendimiento del modelo')
    plt.figure(figsize=(12,8))
    plt.plot(prueba_y_real, label="Real")
    plt.plot(prueba_y_predicted, label="Predicción")
    plt.legend()
    plt.title("Train/Test Dataset")
    
    ## Métricas
    
    rmse = mean_squared_error(y_test_real, y_test_predicted_rnn,squared=False)
    mape = mean_absolute_percentage_error(y_test_real, y_test_predicted_rnn)
    accuracy = 1 - mape
    print('Acurrancy Data Test:',accuracy) #0.92
    print('rmse Data Test :',rmse)
    return(model,scaler)

In [ ]:
# LLamado de la función
model, scaler = main(df)